### Compilation/Generation Benchmark:

#### Requirements for running:
- ZoKrates installed (via install script)


#### Select files:
- Drop all files that are supposed to be benchmarked in the files/ directory in this project

#### Python Imports:

In [21]:
%%capture
import sys
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install csv2md
import os
import math
import matplotlib.pyplot as plt
import numpy as np

#### Iterate Files:

##### Benchmark Settings:
- Each programm will be run N * R times
    - N: Number of executions in one loop
    - R: Number of loop executions

In [22]:
compile_opt = []
setup_opt = []
witness_opt = []
proof_opt = []
constraints_opt = []
compile_unopt = []
setup_unopt = []
witness_unopt = []
proof_unopt = []
constraints_unopt = []
mem_usg = []
files = []

# Set N and R here
%alias_magic benchmark timeit -p "-n 1 -r 1 -o"

def compile_file(file, unopt):
    if unopt:
        cmd = f"./memusg.sh zokrates compile -i files/{file} > temp.txt"
        value = %benchmark !{cmd}
    else:
        cmd = f"./memusg.sh ./zokrates_unoptimized compile -i files/{file} > temp.txt"
        value = %benchmark !{cmd}
    return int(value.best * 1000000)

def setup(unopt):
    if unopt:
        cmd = f"./memusg.sh ./zokrates_unoptimized setup > temp.txt"
        value = %benchmark !{cmd}
    else:
        cmd = f"./memusg.sh zokrates setup > temp.txt"
        value = %benchmark !{cmd}
    return int(value.best * 1000000)

def witness(file, unopt):
    params = get_parameters(file)
    if unopt:
        cmd = f"./memusg.sh ./zokrates_unoptimized compute-witness {params} > temp.txt"
        value = %benchmark !{cmd}
    else:
        cmd = f"./memusg.sh zokrates compute-witness {params} > temp.txt"
        value = %benchmark !{cmd}

    return int(value.best * 1000000)

def proof(unopt):
    if unopt:
        cmd = f"./memusg.sh ./zokrates_unoptimized generate-proof > temp.txt"
        value = %benchmark !{cmd}
    else:
        cmd = f"./memusg.sh zokrates generate-proof > temp.txt"
        value = %benchmark !{cmd}
    return int(value.best * 1000000)

# counts constraints by looking into out.ztf
def count_constraints(unopt):
    f = open("./out.ztf",'r')
    if unopt:
        constraints_unopt.append(len(list(filter(lambda x: "==" in x, f.readlines()))))
    else:
        constraints_opt.append(len(list(filter(lambda x: "==" in x, f.readlines()))))
    
def get_memusg():
    with open('exports/data/memusg_res.txt') as f:
        return [line.rstrip() for line in f]

'''
this method gets the first line of given .zok file and extracts function parameters.
Expected format: 
    -commented out in first line of file with a space after '//'

E.g.

// 337 113569
def main(private field a, field b) -> (field):
  field result = if a * a == b then 1 else 0 fi
  return result
'''
def get_parameters(file):
    with open('files/' + file) as f:
        line = f.readline()
        if "//" in line:
            return "-a" + line.replace("//", "")
        else:
            return ""

# empties files from previous results
def reset_files():
    results = open("exports/data/result.csv", "w")
    results.write("file, compile_opt_μs, memusg_compile, setup_opt_μs, memusg_setup, witness_opt_μs, memusg_witness, proof_opt_μs, memusg_proof, constr_opt, compile_unopt_μs, memusg_compile, setup_unopt_μs, memusg_setup, witness_unopt_μs, memusg_witness, proof_unopt_μs, memusg_proof, constr_unopt\n")
    open("exports/data/memusg_res.txt", 'w').close()
    open("temp.txt", 'w').close()
    
    
def export_data():
    file = open("exports/data/result.csv", "a")
    for i, val in enumerate(compile_opt):
        file.write(
            files[i] + ", " +
            str(val) + ", " + 
            str(mem_usg[i]) + ", " + 
            str(setup_opt[i]) + ", " +
            str(mem_usg[(8 * i) + 1]) + ", " + 
            str(witness_opt[i]) + ", " + 
            str(mem_usg[(8 * i) + 2]) + ", " + 
            str(proof_opt[i]) + ", " + 
            str(mem_usg[(8 * i) + 3]) + ", " + 
            str(constraints_opt[i]) + ", " + 
            str(compile_unopt[i]) + ", " + 
            str(mem_usg[(8 * i) + 4]) + ", " + 
            str(setup_unopt[i]) + ", " + 
            str(mem_usg[(8 * i) + 5]) + ", " + 
            str(witness_unopt[i]) + ", " + 
            str(mem_usg[(8 * i) + 6]) + ", " + 
            str(proof_unopt[i]) + ", " + 
            str(mem_usg[(8 * i) + 7]) + ", " + 
            str(constraints_unopt[i]) + "\n"
        )
    file.close()
    
reset_files()  
for file in os.listdir('./files'):
    print("Compiling optimized...")
    compile_opt.append(compile_file(file, False))
    print("Setup optimized...")
    setup_opt.append(setup(False))
    print("Witness optimized...")
    witness_opt.append(witness(file, False))
    print("Proof optimized...")
    proof_opt.append(proof(False))
    count_constraints(False)
    print("Compiling unoptimized...")
    compile_unopt.append(compile_file(file, True))
    print("Setup unoptimized...")
    setup_unopt.append(setup(True))
    print("Witness unoptimized...")
    witness_unopt.append(witness(file, True))
    print("Proof unoptimized...")
    proof_unopt.append(proof(True))
    count_constraints(True)
    mem_usg = get_memusg()
    
    files.append(file.split('.')[0])

export_data()

Created `%benchmark` as an alias for `%timeit -n 1 -r 1 -o`.
Created `%%benchmark` as an alias for `%%timeit -n 1 -r 1 -o`.
Compiling optimized...
1min 5s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
Setup optimized...
11min 11s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
Witness optimized...
29.6 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
Proof optimized...
1min 19s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
Compiling unoptimized...
1min 8s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
Setup unoptimized...
9min 10s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
Witness unoptimized...
26.5 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
Proof unoptimized...
1min 10s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
Compiling optimized...
1min 45s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
Setup optimized...
21min 26s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
Witnes

In [23]:
!csv2md exports/data/result.csv > exports/data/table.md

with open("exports/data/table.md") as f:
    print(f.read())

| file              |  compile_opt_μs |  memusg_compile |  setup_opt_μs |  memusg_setup |  witness_opt_μs |  memusg_witness |  proof_opt_μs |  memusg_proof |  constr_opt |  compile_unopt_μs |  memusg_compile |  setup_unopt_μs |  memusg_setup |  witness_unopt_μs |  memusg_witness |  proof_unopt_μs |  memusg_proof |  constr_unopt |
| ----------------- | --------------- | --------------- | ------------- | ------------- | --------------- | --------------- | ------------- | ------------- | ----------- | ----------------- | --------------- | --------------- | ------------- | ----------------- | --------------- | --------------- | ------------- | ------------- |
| nft-transfer      |  65609496       |  1991156        |  671130604    |  1745036      |  29562706       |  1106004        |  79028707     |  3123580      |  1422747    |  68826350         |  1768384        |  550970978      |  1551576      |  26452153         |  1025008        |  70542031       |  2820228      |  1208093      |
| ft

#### Compilation and Setup Diagram:


In [ ]:
x = np.arange(len(constraints_opt))  # the label locations
width = 0.15  # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, compile_opt, width, label='Compile', edgecolor='#005eb8', color='#7faedb')
rects2 = ax.bar(x + width/2, setup_opt, width, label='Setup', edgecolor='#ff4c4c', color='#ffa5a5')
ax.set_ylabel('t in μs')
ax.set_xticks(x)
ax.set_xlabel('# of Constraints')
ax.set_xticklabels(constraints_opt)
ax.legend()
ax.semilogy(np.exp(0 / max(setup_opt)))
ax.set_ylim(ymin=1000)
fig.tight_layout()
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.color'] = "#cccccc"
plt.savefig('exports/compile-setup.png')
plt.show()

#### Witness and Proof Diagram:

In [ ]:
x = np.arange(len(constraints_opt))  # the label locations
width = 0.15  # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, witness_opt, width, label='Witness', edgecolor='#005eb8', color='#7faedb')
rects2 = ax.bar(x + width/2, proof_opt, width, label='Proof', edgecolor='#ff4c4c', color='#ffa5a5')
ax.set_ylabel('t in μs')
ax.set_xticks(x)
ax.set_xlabel('# of Constraints')
ax.set_xticklabels(constraints_opt)
ax.legend()
ax.semilogy(np.exp(0 / max(setup_opt)))
ax.set_ylim(ymin=1000)
fig.tight_layout()
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.color'] = "#cccccc"
plt.savefig('exports/witness-proof.png')
plt.show()